In [ ]:
import os
import json
import logging
import uuid
from datetime import datetime, timezone
import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Read from ENV VAR named STATE_MACHINE_ARN, or hardcode as a fallback
SFN_ARN = os.getenv(
    "STATE_MACHINE_ARN",
    "arn:aws:states:us-east-2:178795994454:stateMachine:redshiftstatemachine"
)

# Ensure we talk to the correct region
sfn = boto3.client("stepfunctions", region_name="us-east-2")

def lambda_handler(event, context):
    if not SFN_ARN:
        logger.error("STATE_MACHINE_ARN env var is not set!")
        raise RuntimeError("STATE_MACHINE_ARN env var is not set")

    logger.info("STATE_MACHINE_ARN=%s", SFN_ARN)
    logger.info("Event: %s", json.dumps(event))

    triggered = 0

    for record in event.get("Records", []):
        if record.get("eventSource") != "aws:s3":
            continue

        bucket = record["s3"]["bucket"]["name"]
        key = record["s3"]["object"]["key"]
        size = record["s3"]["object"].get("size")
        etag = record["s3"]["object"].get("eTag")

        if not key.startswith("input/"):
            logger.info("Skipping key %s (not under input/)", key)
            continue

        triggered += 1

        state_input = {
            "bucket": bucket,
            "key": key,
            "size": size,
            "etag": etag,
            "eventTime": record.get("eventTime"),
        }

        exec_name = f"{datetime.now(timezone.utc).strftime('%Y%m%dT%H%M%SZ')}-{uuid.uuid4().hex[:8]}"

        try:
            resp = sfn.start_execution(
                stateMachineArn=SFN_ARN,
                name=exec_name,
                input=json.dumps(state_input),
            )
            logger.info("Started execution: %s", resp["executionArn"])
        except ClientError as e:
            logger.exception("StartExecution failed: %s", e)
            raise

    logger.info("Records that triggered Step Functions: %d", triggered)
    return {"status": "ok", "executions_started": triggered}
